In [2]:
import copy
import datetime
import random
import time

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
from transformers import BertForSequenceClassification, AdamW
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup



In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv('/kaggle/input/reddit/posts.csv', index_col=0)
df = df[['body','is_yta']]
df.dropna
df.head(10)

,body,is_yta
id,,
eiawts,"I'm probably the asshole here, but I figure ma...",nta
eiaw8a,"EDIT: all right, all right. I let you down Red...",yta
eiav15,Sorry this is long\n\nMy parents divorced when...,nta
eiau2i,I’ll try to keep this short. I’m a lawyer. My ...,nta
eiatrs,So to preface I’m a girl(22F) my friend that I...,yta
eiasoj,Context: I’m a 15m who lives in a decently siz...,nta
eias5l,"i’m on mobile\n\nso, i (16f) have a friend (16...",nta
eiaoo9,My sister is doing dual enrollment classes fro...,nta
eiagfd,So I’m going to give some context on a few thi...,nta


In [5]:
df['body']
df[df['body'].isnull()]
df = df.dropna()
df[df['body'].isnull()]

,body,is_yta
id,,


In [6]:
from bs4 import BeautifulSoup
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
df['body'] = df['body'].apply(cleanText)

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:343: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


In [7]:
df.head()

,body,is_yta
id,,
eiawts,"i'm probably the asshole here, but i figure ma...",nta
eiaw8a,"edit: all right, all right. i let you down red...",yta
eiav15,sorry this is long\n\nmy parents divorced when...,nta
eiau2i,i’ll try to keep this short. i’m a lawyer. my ...,nta
eiatrs,so to preface i’m a girl(22f) my friend that i...,yta


In [8]:
train, test = train_test_split(df, test_size=0.25, random_state=42)
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['body']), tags=[r.is_yta]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['body']), tags=[r.is_yta]), axis=1)

In [10]:
train
test

,body,is_yta
id,,
dv1h6q,"at my second house, there is a deaf family acr...",yta
e649gp,me and my brother have a horrible relationship...,nta
dks3ze,"i worked in a bubble tea shop last night, and ...",nta
eg8xfb,christmas came and went. my aunt c does a fami...,nta
dhcd2q,i’ll preface this by saying i typically tip 20...,nta
...,...,...
e7jd79,so my mom has a friend whose youngest daughter...,nta
desesw,throwaway account because my friends follow my...,yta
djpp43,my school's public youtube channel recently po...,yta


In [11]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 11505/11505 [00:00<00:00, 1413541.14it/s]


In [12]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 11505/11505 [00:00<00:00, 1532746.80it/s]


In [18]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors


In [21]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.7011734028683181
Testing F1 score: 0.5803031269073481


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [26]:
print(len(y_test))

3835


In [27]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.6949152542372882
Testing F1 score: 0.5848788996212754


In [30]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.6970013037809648
Testing F1 score: 0.5872829668984914
